# UNSW Pre-processing

In [1]:
cd C:\\Users\\Sharuka Thirimanne\\Desktop\\FYP-ML-IDS

C:\Users\Sharuka Thirimanne\Desktop\FYP-ML-IDS


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
df_train = pd.read_csv('Datasets/UNSW_NB15_training-set.csv')
print('Original dataset duplicates :',df_train.duplicated().sum())

Original dataset duplicates : 74072


In [4]:
df_train.shape

(175341, 43)

In [3]:
def train_preprocess():
    df_train = pd.read_csv('Datasets/UNSW_NB15_training-set.csv')
    dffff = df_train
    num_features = len(df_train.columns) - 1

    print('Number of features before feature selection :',num_features)
    print('Original dataset duplicates :',df_train.duplicated().sum())

    num_features = len(df_train.columns) - 1
    print('Number of features after feature selection :',num_features)
    print('Duplicates after feature selection :',df_train.duplicated().sum())
    
    
    for col in ['proto','service','state']:
        df_train[col] = df_train[col].astype('category')
    
    label1 = 'other_proto'
    others2 = df_train['proto'].value_counts().index[7:]
    df_train['proto'] = df_train['proto'].cat.add_categories([label1])
    df_train['proto'] = df_train['proto'].replace(others2, label1)
    
    label2 = 'other_state'
    others3 = df_train['state'].value_counts().index[4:]
    df_train['state'] = df_train['state'].cat.add_categories([label2])
    df_train['state'] = df_train['state'].replace(others3, label2)
    
    print('Duplicates after category reduction :',df_train.duplicated().sum())
    
    df_train.drop_duplicates(keep='first',inplace=True)
    
    print('Duplicates after duplicate drop :',df_train.duplicated().sum())
    
    return df_train, dffff
df_train, dffff = train_preprocess()

Number of features before feature selection : 42
Original dataset duplicates : 74072
Number of features after feature selection : 42
Duplicates after feature selection : 74072
Duplicates after category reduction : 74295
Duplicates after duplicate drop : 0


In [6]:
df_train.shape

(101046, 43)

In [4]:
print('Original dataset duplicates :',dffff.duplicated().sum())

Original dataset duplicates : 0


In [12]:
print(dffff['proto'].unique())
print(dffff['state'].unique())
print(dffff['service'].unique())

[tcp, udp, arp, ospf, other_proto, unas, sctp, any]
Categories (8, object): [tcp, udp, arp, ospf, other_proto, unas, sctp, any]
[FIN, INT, CON, other_state, REQ]
Categories (5, object): [FIN, INT, CON, other_state, REQ]
[-, ftp, smtp, snmp, http, ..., radius, pop3, dhcp, ssl, irc]
Length: 13
Categories (13, object): [-, ftp, smtp, snmp, ..., pop3, dhcp, ssl, irc]


In [4]:
def test_preprocess():
    df_test = pd.read_csv('Datasets/UNSW_NB15_testing-set.csv')
    df_test.drop(df_test[df_test['state'] == 'ACC'].index, inplace = True)
    df_test.drop(df_test[df_test['state'] == 'CLO'].index, inplace = True)
    print('Original dataset duplicates :',df_test.duplicated().sum())
    
    print('Duplicates after feature selection :',df_test.duplicated().sum())
    df_test.drop_duplicates(keep='first',inplace=True)
    
    for col in ['proto','service','state']:
        df_test[col] = df_test[col].astype('category')
    
    label1 = 'other_proto'
    others4 = df_test['proto'].value_counts().index[7:]
    df_test['proto'] = df_test['proto'].cat.add_categories([label1])
    df_test['proto'] = df_test['proto'].replace(others4, label1)
    
    label2 = 'other_state'
    others5 = df_test['state'].value_counts().index[4:]
    df_test['state'] = df_test['state'].cat.add_categories([label2])
    df_test['state'] = df_test['state'].replace(others5, label2)

    print('Duplicates after category reduction :',df_test.duplicated().sum())
    
    df_test.drop_duplicates(keep='first',inplace=True)
    
    print('Duplicates after duplicate drop :',df_test.duplicated().sum())
    
    return df_test
df_test = test_preprocess()

Original dataset duplicates : 28380
Duplicates after feature selection : 28380
Duplicates after category reduction : 48
Duplicates after duplicate drop : 0


In [5]:
print('Train set shape:',df_train.shape)
print('Test set shape:',df_test.shape)

Train set shape: (101046, 43)
Test set shape: (53899, 43)


In [6]:
print('Train',len(df_train['proto'].unique()))
print('Test',len(df_test['proto'].unique()))
print('=======================================')
print('Train',len(df_train['service'].unique()))
print('Test',len(df_test['service'].unique()))
print('=======================================')
print('Train',len(df_train['state'].unique()))
print('Test',len(df_test['state'].unique()))

Train 8
Test 8
Train 13
Test 13
Train 5
Test 5


In [7]:
limit1 = df_train.shape[1]-1
    
X_train = df_train.iloc[:,0:limit1] # train set features
Y_train = df_train.iloc[:,limit1]
    
limit2 = df_test.shape[1]-1
    
X_test = df_test.iloc[:,0:limit2] # train set features
Y_test = df_test.iloc[:,limit2]

In [8]:
def one_hot_encode(X_train,X_test):
    
    categorical_cols = ['proto','service','state']

    # Training dataset one hot encoding
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    ohe.fit(X_train[categorical_cols])
    array_hot_encoded1 = ohe.transform(X_train[categorical_cols]).toarray()
    data_hot_encoded1 = pd.DataFrame(array_hot_encoded1, index=X_train.index)
    X_train = X_train.drop(columns=categorical_cols)
    X_train = pd.concat([data_hot_encoded1,X_train], axis=1)
    
    print('X_train shape :',X_train.shape)
    
    # Test dataset one hot encoding
    array_hot_encoded2 = ohe.transform(X_test[categorical_cols]).toarray()
    data_hot_encoded2 = pd.DataFrame(array_hot_encoded2, index=X_test.index)
    X_test = X_test.drop(columns=categorical_cols)
    X_test = pd.concat([data_hot_encoded2,X_test], axis=1)
    
    print('X_test shape :',X_test.shape)
    
    return X_train, X_test

X_train, X_test = one_hot_encode(X_train,X_test)

X_train shape : (101046, 65)
X_test shape : (53899, 65)


In [9]:
def scaling(X_train,X_test):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test

X_train, X_test = scaling(X_train,X_test)

In [10]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

Y_train.reset_index(drop=True,inplace=True)
Y_test.reset_index(drop=True,inplace=True)

In [11]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(101046, 65)
(101046,)
(53899, 65)
(53899,)


In [12]:
df_train = pd.concat([X_train,Y_train],axis=1)
df_test = pd.concat([X_test,Y_test],axis=1)

In [13]:
df_test['label'].unique()

array([0, 1], dtype=int64)

In [13]:
print('Train set shape:',df_train.shape)
print('Test set shape:',df_test.shape)

Train set shape: (101046, 66)
Test set shape: (53899, 66)


In [14]:
df_train.groupby('label')['label'].count()

label
0    51890
1    49156
Name: label, dtype: int64

In [15]:
df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

In [16]:
df_test.duplicated().sum()

0

In [17]:
df_train.to_csv('USNW_train-set.csv')
df_test.to_csv('USNW_test-set.csv')